# Summary of Special Class Members

The built-in python stuff like str() and len() will look for certain members on a class.

Since Python is **duck-typed**, there are no interfaces - just implement the members you need.


In [3]:
class MyClass:

    def __init__(self):  # Constructor
        pass

    def __str__(self):  # String representation
        return "MyClass"

    def __repr__(self):  # String representation for debugging
        return "MyClass()"

    def __len__(self):  # Length
        return 42

    def __getitem__(self, index):  # Indexing (could be index, key, whatever)
        return index * 2

    def __setitem__(self, index, value):  # Assignment to an index
        pass

    def __delitem__(self, index):  # Deletion of an index
        pass

    def __iter__(self):  # Iteration
        yield 1
        yield 2
        yield 3

    def __contains__(self, item):  # Membership check
        return True

    def __call__(self):  # Callable behavior
        pass

    def __eq__(self, other):  # Equality comparison
        return True

    def __lt__(self, other):  # Less than comparison
        return True

    def __add__(self, other):  # Addition
        return self

    def __sub__(self, other):  # Subtraction
        return self

    def __and__(self, other):  # bitwise &
        return self

    def __bool__(self):  # conversion to boolean
        return True

    def __int__(self):  # conversion to int
        return 0

    def __enter__(self):  # Context manager enter
        pass

    def __exit__(self, exc_type, exc_val, exc_tb):  # Context manager exit
        pass

    def __getattr__(self, name):  # Accessing undefined attribute
        return None

    def __setattr__(self, name, value):  # Setting attribute
        pass

    def __delattr__(self, name):  # Deleting attribute
        pass

    def __getattribute__(self, name):  # Accessing attribute
        return object.__getattribute__(self, name)

    def __hash__(self):  # Hashing
        return hash(self)

    def __enter__(self):  # Context manager enter
        pass

    def __exit__(self, exc_type, exc_val, exc_tb):  # Context manager exit
        pass

    def __del__(self):  # Destructor
        pass

# Context Manager


In [2]:
class MyContext:

    def __init__(self, message):
        self.message = message

    def __enter__(self):
        print('entering')
        return self  # probably the thing that goes to 'as'

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exiting')
        self.message = None
        if exc_type:
            print(exc_value)
            print(exc_traceback)
            return True  # don't propagate exceptions
        return False  # propagate exceptions


def open_my_context(message):
    return MyContext(message)

In [10]:
# Simple usage of class
with MyContext('hi'):
    pass
print()

# Similar to file I/O API
with open_my_context('hi'):
    pass

entering
exiting

entering
exiting


In [11]:
# Getting a variable for the object
with MyContext('hi') as context:
    print(context.message)
print(context.message)  # variable still exists

entering
hi
exiting
None


In [15]:
# Exception
with MyContext('hi'):
    raise TypeError('uh oh')
    print('unreachable code')

entering
exiting
uh oh


In [4]:
# Multiple contexts without having to nest
with MyContext('c1') as m1, MyContext('c2') as m2:
    print(m1.message)
    print(m2.message)

entering
entering
c1
c2
exiting
exiting


# Iterable


In [5]:
class MyIterator:

    def __init__(self, container):
        self.container = container
        self.index = 0

    def __next__(self):
        if self.index >= len(self.container.data):
            raise StopIteration
        value = self.container.data[self.index]
        self.index += 1
        return value


class MyIterable:

    def __init__(self, data):
        self.data = data

    def __iter__(self):
        return MyIterator(self)


# Usage
my_iterable = MyIterable([1, 2, 3, 4, 5])

for item in my_iterable:
    print(item)
print()

# Lower-level Usage
my_iterable = MyIterable([1, 2, 3, 4, 5])
my_iterator = iter(my_iterable)  # normally the for loop does this
while True:
    try:
        item = next(my_iterator)  # normally the for loop does this
        print(item)
    except StopIteration:  # normally the for loop catches this
        break

1
2
3
4
5

1
2
3
4
5


# Generator

**lazy-loaded**

A generator is a **type of iterable**.


In [6]:
def my_generator():
    print('doing work for 1')
    yield 1  # values provided to next()
    print('doing work for 2')
    yield 2
    print('doing work for 3')
    yield 3
    print('the end')
    return  # ends generation (raises StopIteration)
    yield 4  # never yielded
    # StopIteration would be raised here if no return


generator = my_generator()  # generator function returns iterable
for val in generator:
    print(val)

doing work for 1
1
doing work for 2
2
doing work for 3
3
the end


In [7]:
# Included because a lot of APIs like this pattern
def pass_me_a_generator_function(fn):
    print(list(fn()))


pass_me_a_generator_function(my_generator)

doing work for 1
doing work for 2
doing work for 3
the end
[1, 2, 3]


In [8]:
# Generator comprehension
generator = (i**2 for i in range(3))
for val in generator:
    print(val)

0
1
4


In [9]:
# Convert lazy-loaded generator to in-memory iterable
eager = list(generator)

# Generator from Generator

Iterate over a lazy sequence while **maintaining the laziness**.


In [11]:
def my_generator():
    print('doing work for 1')
    yield 1  # values provided to next()
    print('doing work for 2')
    yield 2
    print('doing work for 3')
    yield 3
    print('the end')
    return  # ends generation (raises StopIteration)
    yield 4  # never yielded


# This would work (and be lazy) even for an eager sequence.
def squares(seq):
    for item in seq:
        yield item**2


gen = my_generator()
sqrs = squares(gen)

print('starting iteration now')
for sqr in sqrs:
    print(sqr)

starting iteration now
doing work for 1
1
doing work for 2
4
doing work for 3
9
the end
